In [1]:
from io import BytesIO
import pycurl,timeit
from multiprocessing import Process
from multiprocessing import Manager

# Performance with proxy

In [2]:
def proxy_latency(prox_type,host,port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.PROXY, f"{host}:{port}")
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = prox_type
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    
    return m


# Performance without proxy

In [3]:
def normal_latency(port,target,return_list):
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(pycurl.URL, target)
    c.setopt(c.WRITEDATA, buffer)
    ts = timeit.default_timer()
    c.perform()
    
    m = {}
    m['prox_type'] = "N/A"
    m['target'] = target
    m['total-time'] = c.getinfo(pycurl.TOTAL_TIME)
    m['namelookup-time'] = c.getinfo(pycurl.NAMELOOKUP_TIME)
    m['connect-time'] = c.getinfo(pycurl.CONNECT_TIME)
    m['pretransfer-time'] = c.getinfo(pycurl.PRETRANSFER_TIME)
    m['starttransfer-time'] = c.getinfo(pycurl.STARTTRANSFER_TIME)
    
    return_list.append(m)
    
    c.close()
    

# Setup

In [40]:
host = "127.0.0.1"
port = "8080"
range_k = 100



# target = "https://github.com/"

# target = "http://www.cs.cmu.edu/~prs/bio.html"


target = "http://www.cs.tufts.edu/comp/112/"

# Without proxy

## 1. One time

In [41]:
return_list = []
normal_latency(port,target,return_list)
print(return_list[0])

{'prox_type': 'N/A', 'target': 'http://www.cs.tufts.edu/comp/112/', 'total-time': 0.059161, 'namelookup-time': 0.003783, 'connect-time': 0.005431, 'pretransfer-time': 0.005512, 'starttransfer-time': 0.059105}


## 2. Average time (100 request)

In [42]:
total = 0
return_list = []
for n in range(range_k):
    result = normal_latency(port,target,return_list)

for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.013562310000000001

# Our project - select + pthread

In [31]:
prox_type = "select"

## 1. One time without cache

In [32]:
return_list = []
proxy_latency(prox_type,host,port,target,return_list)
print(return_list[0])

{'prox_type': 'select', 'target': 'http://www.cs.cmu.edu/~prs/bio.html', 'total-time': 0.057463, 'namelookup-time': 2.5e-05, 'connect-time': 0.000125, 'pretransfer-time': 0.000148, 'starttransfer-time': 0.057411}


## 2. Average time with cache (100 reqests)

In [33]:
total = 0
return_list = []
for n in range(range_k):
    proxy_latency(prox_type,host,port,target,return_list)

    
for result in return_list: 
    total+= result["total-time"]
    
total/range_k

0.07465004999999998

# Simulate multi users - multiprocesses

In [35]:
range_k = 100

## Without proxy - http, 50 request

In [38]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=normal_latency,args=(port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k    

1.5132486600000001

## Proxy - http with cache, 50 request

In [39]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=proxy_latency,args=(prox_type,host,port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k  

3.9978136000000006

In [13]:
target = "https://github.com/"
range_k = 200

## Without proxy - https, 50 request

In [14]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=normal_latency,args=(port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k    

1.1721750599999994

## Proxy - https withoutcache, 50 request

In [15]:
manager = Manager()
return_list = manager.list()
jobs = []
total = 0

for i in range(range_k):
    p = Process(target=proxy_latency,args=(prox_type,host,port,target,return_list))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

for item in return_list:
    total+=item["total-time"]

total/range_k 

Process Process-405:
Traceback (most recent call last):
Process Process-406:
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-407:
Traceback (most recen

  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-527:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-531:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/pytho

  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-482:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-498:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/pytho

  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-586:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-554:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", l

pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-420:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-441:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL rou

...

  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-434:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-429:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", l

pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-592:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-589:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL rou

Process Process-501:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-502:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-504:
Traceback (most recen

  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-549:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sixiangz/anaconda3/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-8cd7f3f5b5ea>", line 8, in proxy_latency
    c.perform()
pycurl.error: (35, 'error:1408F10B:SSL routines:ssl3_get_record:wrong version number')
Process Process-543:
Traceback (most recent call last):
  File "/home/sixiangz/anaconda3/lib/pytho

0.0

In [16]:
# 10: https noproxy: 0.21 select+pthread: 0.33 select: 0.48  
# 50: https noproxy: 0.42 select+pthread: 1.35 select: 2.11  
# 100: https noproxy: 0.78 select+pthread: 2.24 select: 3.32 
# 200: https noproxy: 1.26 select+pthread: 4.12 select: 7.11 

In [17]:
# Single reqest
# https noproxy: 0.17 select+pthread: 0.18 select: 0.21